# Build a Neural network with Hyper-parameter fine tuning model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import GridSearchCV

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0


In [2]:
# Define the CNN model function
def create_model(activation='relu', optimizer='adam'):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation=activation, input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation=activation))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation=activation))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Define hyperparameters for grid search
param_grid = {
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop']
}

# Perform Grid Search
best_accuracy = 0
best_params = {}

In [3]:
for activation in param_grid['activation']:
    for optimizer in param_grid['optimizer']:
        print(f"\nTraining with activation={activation}, optimizer={optimizer}")
        
        # Create and compile the model
        model = create_model(activation=activation, optimizer=optimizer)
        
        # Train the model
        model.fit(train_images, train_labels, epochs=10, batch_size=64, verbose=0)
        
        # Evaluate the model
        _, accuracy = model.evaluate(test_images, test_labels, verbose=0)
        
        # Update best parameters if the current model is better
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'activation': activation, 'optimizer': optimizer}



Training with activation=relu, optimizer=adam

Training with activation=relu, optimizer=rmsprop

Training with activation=tanh, optimizer=adam

Training with activation=tanh, optimizer=rmsprop


In [4]:
# Print results
print(f"\nBest Accuracy: {best_accuracy:.4f} using {best_params}")


Best Accuracy: 0.7032 using {'activation': 'relu', 'optimizer': 'adam'}


# 

# 

# Build an image classifier model with Pytorch

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [6]:
pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/74/07/edce54779f5c3fe8ab8390eafad3d7c8190fce68f922a254ea77f4a94a99/torch-2.1.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB 1.7 MB/s eta 0:01:57
   ---------------------------------------- 0.1/192.3 MB 1.7 MB/s eta 0:01:53
   ---------------------------------------- 0.3/192.3 MB 2.7 MB/s eta 0:01:11
   ---------------------------------------- 0.4/192.3 MB 2.4 MB/s eta 0:01:21
   ---------------------------------------- 0.6/192.3 MB 2.8 MB/s eta 0:01:09
   ---------------------------------------- 0.8/192.3 MB 3.0 MB/s eta 0:01:04
   ---------------------------------------- 1.0/192.3 MB 3.3 MB/s eta 0:00:58
   ---------------------------------------- 1.3/192.3 MB 3.5 MB/s eta 0:00:55
   ---------------------------------------- 1.6/192.3 MB 3.8 MB/s eta 0:00:51
   --------------------

In [8]:
pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/20/ac/ab6f42af83349e679b03c9bb18354740c6b58b17dba329fb408730230584/torchvision-0.16.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.3 MB 1.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.3 MB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.3 MB 1.5 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.3 MB 2.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.3 MB 2.8 MB/s eta 0:00:01
   --------------------------- ------------ 0.9/1.3 MB 3.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.1/1.3 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [11]:
# Download CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [01:21<00:00, 2084676.98it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
# Instantiate the model, loss function, and optimizer
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [16]:
# Training the model
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.148
[1,  4000] loss: 1.838
[1,  6000] loss: 1.660
[1,  8000] loss: 1.554
[1, 10000] loss: 1.525
[1, 12000] loss: 1.456
[2,  2000] loss: 1.400
[2,  4000] loss: 1.374
[2,  6000] loss: 1.339
[2,  8000] loss: 1.313
[2, 10000] loss: 1.294
[2, 12000] loss: 1.247
[3,  2000] loss: 1.196
[3,  4000] loss: 1.189
[3,  6000] loss: 1.191
[3,  8000] loss: 1.177
[3, 10000] loss: 1.180
[3, 12000] loss: 1.174
[4,  2000] loss: 1.092
[4,  4000] loss: 1.091
[4,  6000] loss: 1.094
[4,  8000] loss: 1.085
[4, 10000] loss: 1.084
[4, 12000] loss: 1.093
[5,  2000] loss: 0.997
[5,  4000] loss: 1.015
[5,  6000] loss: 1.038
[5,  8000] loss: 1.000
[5, 10000] loss: 1.026
[5, 12000] loss: 1.027
Finished Training


In [17]:
# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [18]:
print('Accuracy on the test set: %d %%' % (100 * correct / total))

Accuracy on the test set: 62 %
